In [2]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2 
import io

# Clean Trail Data

In [23]:
# Load trail info data
df_trail = pd.read_pickle('/Users/briangraham/insight/trailrec/data/trail_info_combined.pickle')

# Eliminate trails without descriptions
df_trail = df_trail[df_trail['description'].notnull()]

# subset trail data, eliminating uncommonly reported features with mostly NaN
df_trail = df_trail[['Altitude change', 'Altitude end', 'Altitude max', 'Altitude min',
       'Altitude start', 'Avg reverse time', 'Avg time', 'Bike type',
       'Difficulty rating', 'Direction', 'Distance climb', 'Distance down',
       'Global Ranking', 'Grade', 'Grade max', 'Grade min', 'Physical rating',
       'Ride in rain', 'Riding area', 'TTFs on trail', 'Trail Usage',
       'Trail type', 'avg_time', 'city', 'climb', 'descent', 'description',
       'distance', 'latitude', 'longitude', 'rating', 'state', 'votes',
       'Climb Difficulty', 'Distance flat']]
df_trail['votes'] = df_trail['votes'].str.replace(' votes','')
df_trail['votes'] = pd.to_numeric(df_trail['votes'])
df_trail['rating'] = df_trail['rating'].str.replace('Avg:','')
df_trail['rating'] = pd.to_numeric(df_trail['rating'])
df_trail['latitude'] = pd.to_numeric(df_trail['latitude'])
df_trail['longitude'] = pd.to_numeric(df_trail['longitude'])

## Have to go through this data and remove units from strings and convert to numbers

In [83]:
df_trail.head()

,Altitude change,Altitude end,Altitude max,Altitude min,Altitude start,Avg reverse time,Avg time,Bike type,Difficulty rating,Direction,...,descent,description,distance,latitude,longitude,rating,state,votes,Climb Difficulty,Distance flat
trail_id,,,,,,,,,,,,,,,,,,,,,
1-87-dh,"-1,508 ft",349 ft,"1,857 ft",349 ft,"1,857 ft",00:01:40,00:12:58,"DH, AM",Black Diamondrate,Downhill Primary,...,"-1,513 ft",Sustained steep and fast DH trail often used a...,2 miles,48.82592,-123.762540,4.55,BC,13,NaN,NaN
10-dollar,-191 ft,"2,446 ft","2,636 ft","2,446 ft","2,636 ft",00:15:47,00:09:03,"DH, AM, XC",Bluerate,NaN,...,-215 ft,One of the original Pidherny trails.,"2,226 ft",53.98968,-122.846810,3.50,BC,8,NaN,76 ft
10-km,-694 ft,"4,931 ft","5,625 ft","4,918 ft","5,625 ft",NaN,00:06:44,DH,Black Diamondrate,NaN,...,-716 ft,The 10K trail is a good early season starting ...,"2,840 ft",51.28040,-116.893354,3.20,BC,5,NaN,NaN
115th-ave-connector,-46 ft,"1,054 ft",NaN,"1,054 ft","1,100 ft",NaN,NaN,XC,Whiterate,NaN,...,-54 ft,Short connector to access the Osoyoos Canal.,"1,417 ft",49.03372,-119.488330,1.00,BC,1,Green,48 ft
147,-0 ft,"2,638 ft","2,638 ft","2,635 ft","2,638 ft",00:07:23,00:01:17,"DH, AM, Fat",Bluerate,Both Directions,...,-3 ft,"Short, technical trail that connects The Swamp...",324 ft,53.87815,-122.828430,4.00,BC,1,Green,64 ft


# Eliminate comments that do not match this trail subset

In [62]:
df_comments = pd.read_pickle('/Users/briangraham/insight/trailrec/data/trail_comments_combined.pickle')
df_comments_filtered = df_comments.join(df_trail,how='right')
df_comments_filtered = df_comments_filtered[['comment_votes','user','comment_text']]
df_comments_filtered = df_comments_filtered.dropna()

In [63]:
df_comments_filtered.head()

,comment_votes,user,comment_text
trail_id,,,
19th-hole,+ 1,blackcombterry,What's 19th Hole like below the Upper Legacy c...
2-3481,+ 0,atfarley,Just a road
2001,+ 1,amowat,Still some good ladders but alot gone. Big roc...
2001,+ 0,MtbSince84,Closed due to logging.
280-s,+ 0,HexAngel,Gently insistent ascent even this unfit geezer...


# Eliminate ridelogs that do not match this trail subset

In [66]:
df_ridelogs = pd.read_pickle('/Users/briangraham/insight/trailrec/data/trail_ridelogs_combined.pickle')
df_ridelogs_match = ridelogs.join(df_trail,how='right')
df_ridelogs_match = df_ridelogs_match[['ride_date','rider_url','rider_name','rider_state','rider_city','ride_difficulty_string']]
df_ridelogs_match.head()

,ride_date,rider_url,rider_name,rider_state,rider_city,ride_difficulty_string
trail_id,,,,,,
1-87-dh,Jul 5,https://www.trailforks.com/profile/BCpov/,BCpov,BC,Duncan,Very Difficult / Black Diamond
1-87-dh,Jul 5,https://www.trailforks.com/profile/canadaka/,canadaka,BC,Duncan,Very Difficult / Black Diamond
1-87-dh,May 27,https://www.trailforks.com/profile/campasternak/,campasternak,BC,Duncan,Very Difficult / Black Diamond
1-87-dh,May 21,https://www.trailforks.com/profile/campasternak/,campasternak,BC,Duncan,Very Difficult / Black Diamond
1-87-dh,May 20,https://www.trailforks.com/profile/superlightr...,superlightracer,BC,Duncan,Very Difficult / Black Diamond


In [67]:
df_ridelogs_match.shape

(1266757, 6)

# Save to Postgres

In [70]:
# Define a database name (we're using a dataset on births, so we'll call it birth_db)
# Set your postgres username
dbname = 'trailrec'
username = 'briangraham' # change this to your username

In [71]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://briangraham@localhost/trailrec


In [72]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [73]:
## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
df_trail.to_sql('trails', engine, if_exists='replace')
df_comments_filtered.to_sql('comments', engine, if_exists='replace')
df_ridelogs_match.to_sql('ridelogs', engine, if_exists='replace')

In [3]:
# Connect to make queries using psycopg2
conn = psycopg2.connect("host=localhost dbname=trailrec user=briangraham")
cur = conn.cursor()

In [16]:
# query:
sql_query = """
SELECT * FROM trails;
"""
birth_data_from_sql = pd.read_sql_query(sql_query,conn)
birth_data_from_sql.tail()

,trail_id,Altitude change,Altitude end,Altitude max,Altitude min,Altitude start,Avg reverse time,Avg time,Bike type,Difficulty rating,...,descent,description,distance,latitude,longitude,rating,state,votes,Climb Difficulty,Distance flat
2797,zig-zag-27113,-71 ft,732 ft,804 ft,731 ft,803 ft,None,00:05:14,"DH, AM",Black Diamondrate,...,-73 ft,Well built DH style trail with optional ride a...,"2,107 ft",48.409780,-123.679800,5.00,BC,7,None,332 ft
2798,ziggy-8689,-416 ft,"3,186 ft","3,608 ft","3,186 ft","3,602 ft",00:22:46,00:05:49,"AM, XC",Bluerate,...,-432 ft,"In the Spring and early Summer, you will get m...","4,833 ft",50.086962,-120.854856,4.65,BC,3,None,None
2799,zolapalusa-extra-crispy,-711 ft,"2,952 ft","3,781 ft","2,952 ft","3,663 ft",None,00:15:54,"AM, XC",Black Diamondrate,...,-931 ft,This trail is for hiking and mountain biking. ...,1 miles,50.840090,-122.881470,3.00,BC,1,None,"1,368 ft"
2800,zoot-allures,-31 ft,"2,262 ft","2,292 ft","2,262 ft","2,292 ft",00:01:33,00:00:58,"AM, XC",Bluerate,...,-34 ft,This trail is one of many riding options surro...,599 ft,50.135020,-122.943790,4.35,BC,6,None,165 ft
2801,zoso,-11 ft,457 ft,476 ft,446 ft,468 ft,00:02:16,00:01:56,"AM, XC, Fat",Bluerate,...,-34 ft,"Twisty, rolly, mildly technical, and open. Nic...",968 ft,48.521230,-123.459380,3.00,BC,1,None,218 ft


In [12]:
birth_data_from_sql['description'].iloc[0]

'One of the original Pidherny trails.'